In [17]:
import argparse
from midi2audio import FluidSynth
import tensorflow as tf
import os
import numpy as np
from model import GenreLSTM
import mido
import midi_util
from mido import MidiFile
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

88


In [3]:
def setup_dir():

    print('[*] Setting up directory...')

    main_path = './runs'
    current_run = main_path

    files_path = './data'
#     files_path = os.path.join(files_path, args.data_set)

    x_path = os.path.join(files_path, 'inputs')
    y_path = os.path.join(files_path, 'velocities')
    eval_path = os.path.join(files_path, 'eval')

    model_path = os.path.join(current_run, 'model')
    logs_path = os.path.join(current_run, 'tmp')
    png_path = os.path.join(current_run, 'png')
    pred_path = os.path.join(current_run, 'predictions')

    if not os.path.exists(current_run):
        os.makedirs(current_run)
    if not os.path.exists(logs_path):
        os.makedirs(logs_path)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    if not os.path.exists(png_path):
        os.makedirs(png_path)
    if not os.path.exists(pred_path):
        os.makedirs(pred_path)

    dirs = {
            'main_path': main_path,
            'current_run': current_run,
            'model_path': model_path,
            'logs_path': logs_path,
            'png_path': png_path,
            'eval_path': eval_path,
            'pred_path': pred_path,
            'x_path': x_path,
            'y_path': y_path
        }

#     print main_path
#     print current_run
#     print model_path
#     print logs_path
#     print png_path
#     print eval_path
#     print x_path
#     print y_path
    return dirs

In [4]:
setup_dir()

[*] Setting up directory...


{'current_run': './runs',
 'eval_path': './data/eval',
 'logs_path': './runs/tmp',
 'main_path': './runs',
 'model_path': './runs/model',
 'png_path': './runs/png',
 'pred_path': './runs/predictions',
 'x_path': './data/inputs',
 'y_path': './data/velocities'}

In [5]:
def load_training_data(x_path, y_path, genre):
    X_data = []
    Y_data = []
    names = []
    print('[*] Loading data...')

    x_path = os.path.join(x_path, genre)
    y_path = os.path.join(y_path, genre)

    for i, filename in enumerate(os.listdir(x_path)):
#         print(filename)
        if filename.split('.')[-1] == 'npy':
            names.append(filename)

    for i, filename in enumerate(names):
        abs_x_path = os.path.join(x_path,filename)
        abs_y_path = os.path.join(y_path,filename)
#         print(abs_x_path)
#         print(abs_y_path)
        loaded_x = np.load(abs_x_path)

        X_data.append(loaded_x)

        loaded_y = np.load(abs_y_path)
        loaded_y = loaded_y/127
        Y_data.append(loaded_y)
        assert X_data[i].shape[0] == Y_data[i].shape[0]
    return X_data, Y_data

In [6]:
dirs = setup_dir()
data = {}
data["classical"] = {}
data["jazz"] = {}

c_train_X , c_train_Y = load_training_data(dirs['x_path'], dirs['y_path'], "classical")

data["classical"]["X"] = c_train_X
data["classical"]["Y"] = c_train_Y

j_train_X , j_train_Y = load_training_data(dirs['x_path'], dirs['y_path'], "jazz")

data["jazz"]["X"] = j_train_X
data["jazz"]["Y"] = j_train_Y

[*] Setting up directory...
[*] Loading data...
[*] Loading data...


In [7]:
print(len(data))
print(len(data['classical']['X']))
print(len(data['classical']['Y']))
print(len(data['jazz']['X']))
print(len(data['jazz']['Y']))

2
183
183
349
349


In [8]:
tf.logging.set_verbosity(tf.logging.ERROR)
network  = GenreLSTM(dirs, input_size=176, mini=True)
network.prepare_model()

[*] Preparing unidirectional dynamic RNN...


In [9]:
network.train(data)

[*] Initiating training...
[*] Epoch 0
	Batch 1/23, Training MSE for Classical batch: 0.014408159
	Batch 2/23, Training MSE for Classical batch: 0.011220982
	Batch 3/23, Training MSE for Classical batch: 0.012576969
	Batch 4/23, Training MSE for Classical batch: 0.008711634
	Batch 5/23, Training MSE for Classical batch: 0.009072772
	Batch 6/23, Training MSE for Classical batch: 0.007990012
	Batch 7/23, Training MSE for Classical batch: 0.006834066
	Batch 8/23, Training MSE for Classical batch: 0.008629955
	Batch 9/23, Training MSE for Classical batch: 0.005202761
	Batch 10/23, Training MSE for Classical batch: 0.005816566
	Batch 11/23, Training MSE for Classical batch: 0.005076796
	Batch 12/23, Training MSE for Classical batch: 0.006389843
	Batch 13/23, Training MSE for Classical batch: 0.005123571
	Batch 14/23, Training MSE for Classical batch: 0.003902835
	Batch 15/23, Training MSE for Classical batch: 0.004823435
	Batch 16/23, Training MSE for Classical batch: 0.005110243
	Batch 17/

	Batch 2/23, Training MSE for Classical batch: 0.004180948
	Batch 3/23, Training MSE for Classical batch: 0.004668867
	Batch 4/23, Training MSE for Classical batch: 0.002480398
	Batch 5/23, Training MSE for Classical batch: 0.003354232
	Batch 6/23, Training MSE for Classical batch: 0.003455799
	Batch 7/23, Training MSE for Classical batch: 0.003807441
	Batch 8/23, Training MSE for Classical batch: 0.002503779
	Batch 9/23, Training MSE for Classical batch: 0.003820087
	Batch 10/23, Training MSE for Classical batch: 0.003056162
	Batch 11/23, Training MSE for Classical batch: 0.002549254
	Batch 12/23, Training MSE for Classical batch: 0.002429293
	Batch 13/23, Training MSE for Classical batch: 0.005817722
	Batch 14/23, Training MSE for Classical batch: 0.004328644
	Batch 15/23, Training MSE for Classical batch: 0.003298761
	Batch 16/23, Training MSE for Classical batch: 0.003603855
	Batch 17/23, Training MSE for Classical batch: 0.003881484
	Batch 18/23, Training MSE for Classical batch: 

	Batch 6/23, Training MSE for Classical batch: 0.002711056
	Batch 7/23, Training MSE for Classical batch: 0.002652715
	Batch 8/23, Training MSE for Classical batch: 0.002616968
	Batch 9/23, Training MSE for Classical batch: 0.003429912
	Batch 10/23, Training MSE for Classical batch: 0.002590345
	Batch 11/23, Training MSE for Classical batch: 0.002139797
	Batch 12/23, Training MSE for Classical batch: 0.003155287
	Batch 13/23, Training MSE for Classical batch: 0.004091662
	Batch 14/23, Training MSE for Classical batch: 0.003549508
	Batch 15/23, Training MSE for Classical batch: 0.003816020
	Batch 16/23, Training MSE for Classical batch: 0.002656111
	Batch 17/23, Training MSE for Classical batch: 0.003350840
	Batch 18/23, Training MSE for Classical batch: 0.002414175
	Batch 19/23, Training MSE for Classical batch: 0.004533921
	Batch 20/23, Training MSE for Classical batch: 0.004176313
	Batch 21/23, Training MSE for Classical batch: 0.001620525
	Batch 22/23, Training MSE for Classical bat

	Batch 10/23, Training MSE for Classical batch: 0.003336482
	Batch 11/23, Training MSE for Classical batch: 0.003552153
	Batch 12/23, Training MSE for Classical batch: 0.002986997
	Batch 13/23, Training MSE for Classical batch: 0.003113153
	Batch 14/23, Training MSE for Classical batch: 0.002247173
	Batch 15/23, Training MSE for Classical batch: 0.001457607
	Batch 16/23, Training MSE for Classical batch: 0.003462615
	Batch 17/23, Training MSE for Classical batch: 0.002822703
	Batch 18/23, Training MSE for Classical batch: 0.002215438
	Batch 19/23, Training MSE for Classical batch: 0.003804146
	Batch 20/23, Training MSE for Classical batch: 0.005169577
	Batch 21/23, Training MSE for Classical batch: 0.001869082
	Batch 22/23, Training MSE for Classical batch: 0.002152848
	Batch 23/23, Training MSE for Classical batch: 0.001740024
	Batch 1/44, Training MSE for Jazz batch: 0.002927497
	Batch 2/44, Training MSE for Jazz batch: 0.003629190
	Batch 3/44, Training MSE for Jazz batch: 0.00318620

	Batch 14/23, Training MSE for Classical batch: 0.002844913
	Batch 15/23, Training MSE for Classical batch: 0.003179437
	Batch 16/23, Training MSE for Classical batch: 0.002733228
	Batch 17/23, Training MSE for Classical batch: 0.001884937
	Batch 18/23, Training MSE for Classical batch: 0.001880402
	Batch 19/23, Training MSE for Classical batch: 0.002491071
	Batch 20/23, Training MSE for Classical batch: 0.002094728
	Batch 21/23, Training MSE for Classical batch: 0.002317118
	Batch 22/23, Training MSE for Classical batch: 0.002047329
	Batch 23/23, Training MSE for Classical batch: 0.001815544
	Batch 1/44, Training MSE for Jazz batch: 0.002273623
	Batch 2/44, Training MSE for Jazz batch: 0.003162743
	Batch 3/44, Training MSE for Jazz batch: 0.002390438
	Batch 4/44, Training MSE for Jazz batch: 0.003750829
	Batch 5/44, Training MSE for Jazz batch: 0.003762859
	Batch 6/44, Training MSE for Jazz batch: 0.003004871
	Batch 7/44, Training MSE for Jazz batch: 0.003260226
	Batch 8/44, Training 

...

	Batch 33/44, Training MSE for Jazz batch: 0.002652422
	Batch 34/44, Training MSE for Jazz batch: 0.003183149
	Batch 35/44, Training MSE for Jazz batch: 0.003079337
	Batch 36/44, Training MSE for Jazz batch: 0.002580741
	Batch 37/44, Training MSE for Jazz batch: 0.003785433
	Batch 38/44, Training MSE for Jazz batch: 0.002240526
	Batch 39/44, Training MSE for Jazz batch: 0.003213946
	Batch 40/44, Training MSE for Jazz batch: 0.002488824
	Batch 41/44, Training MSE for Jazz batch: 0.002191727
	Batch 42/44, Training MSE for Jazz batch: 0.003165736
	Batch 43/44, Training MSE for Jazz batch: 0.003540059
	Batch 44/44, Training MSE for Jazz batch: 0.001729429
[*] Average Training MSE for Classical epoch 32: 0.002106773
[*] Average Training MSE for Jazz epoch 32: 0.002759643
[*] Saving checkpoint...
[*] Model saved in file: ./runs/model/model-e32.ckpt
[*] Epoch 33
	Batch 1/23, Training MSE for Classical batch: 0.001522346
	Batch 2/23, Training MSE for Classical batch: 0.001523993
	Batch 3/23, T

	Batch 37/44, Training MSE for Jazz batch: 0.003427071
	Batch 38/44, Training MSE for Jazz batch: 0.002673580
	Batch 39/44, Training MSE for Jazz batch: 0.001839223
	Batch 40/44, Training MSE for Jazz batch: 0.002843868
	Batch 41/44, Training MSE for Jazz batch: 0.002022486
	Batch 42/44, Training MSE for Jazz batch: 0.002273967
	Batch 43/44, Training MSE for Jazz batch: 0.003951008
	Batch 44/44, Training MSE for Jazz batch: 0.003187718
[*] Average Training MSE for Classical epoch 34: 0.002049700
[*] Average Training MSE for Jazz epoch 34: 0.002825779
[*] Saving checkpoint...
[*] Model saved in file: ./runs/model/model-e34.ckpt
[*] Epoch 35
	Batch 1/23, Training MSE for Classical batch: 0.001419231
	Batch 2/23, Training MSE for Classical batch: 0.003126594
	Batch 3/23, Training MSE for Classical batch: 0.001876330
	Batch 4/23, Training MSE for Classical batch: 0.002182238
	Batch 5/23, Training MSE for Classical batch: 0.002044054
	Batch 6/23, Training MSE for Classical batch: 0.00239524

	Batch 41/44, Training MSE for Jazz batch: 0.002533953
	Batch 42/44, Training MSE for Jazz batch: 0.003001045
	Batch 43/44, Training MSE for Jazz batch: 0.003469961
	Batch 44/44, Training MSE for Jazz batch: 0.002578535
[*] Average Training MSE for Classical epoch 36: 0.002049519
[*] Average Training MSE for Jazz epoch 36: 0.002794892
[*] Saving checkpoint...
[*] Model saved in file: ./runs/model/model-e36.ckpt
[*] Epoch 37
	Batch 1/23, Training MSE for Classical batch: 0.001861495
	Batch 2/23, Training MSE for Classical batch: 0.001678451
	Batch 3/23, Training MSE for Classical batch: 0.002697831
	Batch 4/23, Training MSE for Classical batch: 0.003261456
	Batch 5/23, Training MSE for Classical batch: 0.002770151
	Batch 6/23, Training MSE for Classical batch: 0.004803679
	Batch 7/23, Training MSE for Classical batch: 0.002036425
	Batch 8/23, Training MSE for Classical batch: 0.002356129
	Batch 9/23, Training MSE for Classical batch: 0.003314544
	Batch 10/23, Training MSE for Classical 

[*] Model saved in file: ./runs/model/model-e38.ckpt
[*] Epoch 39
	Batch 1/23, Training MSE for Classical batch: 0.001907440
	Batch 2/23, Training MSE for Classical batch: 0.001671962
	Batch 3/23, Training MSE for Classical batch: 0.002137629
	Batch 4/23, Training MSE for Classical batch: 0.002558170
	Batch 5/23, Training MSE for Classical batch: 0.001372711
	Batch 6/23, Training MSE for Classical batch: 0.001459689
	Batch 7/23, Training MSE for Classical batch: 0.005940316
	Batch 8/23, Training MSE for Classical batch: 0.001787449
	Batch 9/23, Training MSE for Classical batch: 0.003107814
	Batch 10/23, Training MSE for Classical batch: 0.002409330
	Batch 11/23, Training MSE for Classical batch: 0.001513469
	Batch 12/23, Training MSE for Classical batch: 0.002952918
	Batch 13/23, Training MSE for Classical batch: 0.001636958
	Batch 14/23, Training MSE for Classical batch: 0.001859634
	Batch 15/23, Training MSE for Classical batch: 0.002686155
	Batch 16/23, Training MSE for Classical ba

KeyboardInterrupt: 

In [10]:
network.load("a", path="./runs/model/model-e40.ckpt")

 [*] Loading checkpoint...


In [11]:
input_dir = "./data/inputs/"
output_dir = "./data/velocities/"
jazz_names = []
classical_names = []

In [12]:
for i, filename in enumerate(os.listdir(os.path.join(input_dir, "jazz"))):
    if filename.split(".")[-1] == "npy":
        jazz_names.append(filename)

        
for i, filename in enumerate(os.listdir(os.path.join(input_dir, "classical"))):
    if filename.split(".")[-1] == "npy":
        classical_names.append(filename)


In [15]:
print(len(jazz_names))
print(jazz_names[0])

349
Remember - solo jazz piano.mid.npy


In [19]:
for filename in classical_names:
    print filename
    name = filename.split(".")[0] + ".mid"
    classical_input = os.path.join("./data/inputs/classical", filename)
    classical_velocities = os.path.join("./data/velocities/classical", filename)
    c_error, c_out, j_out, e_out, out_list = network.predict(classical_input, classical_velocities)
#     plot_evaluation(1, "n2", c_out, j_out, e_out, out_list)
    mid = MidiFile(os.path.join("./predict/original/classical/", name))
    classical = midi_util.stylify(mid, c_out[-1], 4)
    classical.save(os.path.join("./predict/original_classical/classical/", name))
    
    mid = MidiFile(os.path.join("./predict/original/classical/", name))
    jazz = midi_util.stylify(mid, j_out[-1], 4)
    jazz.save(os.path.join("./predict/original_classical/jazz/", name))

LEE_J03.mid.npy
KARYAG07.mid.npy
beethoven_opus22_1_format0.mid.npy
mendel_op19_1_format0.mid.npy
sonatina op36 n3 1mov.mid.npy
deb_pass_format0.mid.npy
chpn_op25_e1_format0.mid.npy
burg_gewitter_format0.mid.npy
haydn_9_1_format0.mid.npy
KARYAG06.mid.npy
FONG01.mid.npy
KRASNI09.mid.npy
rac_op3_2_format0.mid.npy
LEE_K07.mid.npy
mendel.mid.npy
burg_spinnerlied_format0.mid.npy
YOO04.mid.npy
mz_570_2_format0.mid.npy
Piano Sonata n10 2mov.mid.npy
schubert_D850_4_format0.mid.npy
ISHIDA01.mid.npy
ty_oktober_format0.mid.npy
mz_570_3_format0.mid.npy
chno1501.mid.npy
schubert_d935_1_format0.mid.npy
KRASNI07.mid.npy
schumann.mid.npy
chpn-p23_format0.mid.npy
hay-p52a.mid.npy
ty_september_format0.mid.npy
SINKEV09.mid.npy
debussy_cc_5_format0.mid.npy
chpn_op10_e01_format0.mid.npy
men-gon3.mid.npy
Irrational-Rose.mid.npy
KARYAG04.mid.npy
mikhailoff02.mid.npy
02. Christe Eleison .mid.npy


IOError: [Errno 2] No such file or directory: './predict/original/classical/02.mid'

In [21]:
for filename in jazz_names:
    print filename
    name = filename.split(".")[0] + ".mid"
    jazz_input = os.path.join("./data/inputs/jazz", filename)
    jazz_velocities = os.path.join("./data/velocities/jazz", filename)
    c_error, c_out, j_out, e_out, out_list = network.predict(jazz_input, jazz_velocities)
    mid = MidiFile(os.path.join("./predict/original/jazz/", name))
    classical = midi_util.stylify(mid, c_out[-1], 4)
#     plot_evaluation(0, "filename", c_out, j_out, e_out, out_list)
    classical.save(os.path.join("./predict/original_jazz/classical/", name))
    mid = MidiFile(os.path.join("./predict/original/jazz/", name))
    jazz = midi_util.stylify(mid, j_out[-1], 4)
    jazz.save(os.path.join("./predict/original_jazz/jazz/", name))


Remember - solo jazz piano.mid.npy
alltheth.mid.npy
StrangeMeadowlark.mid.npy
If_bell.mid.npy
TheOddCpl.mid.npy
I Thought About You (Doug McKenzie).mid.npy
simplelife.mid.npy
couldwrite.mid.npy
The Way You look trio.mid.npy
atIknew.mid.npy
comesun.mid.npy
bethlehem.mid.npy
periscope.mid.npy
conalma.mid.npy
Blameiton.mid.npy
StWeather.mid.npy
Warmvalleysolo.mid.npy
grooving_hard.mid.npy
merryxmas.mid.npy
mwv_blues.mid.npy
ishouldcare2.mid.npy
BigMess.mid.npy
shinystockings.mid.npy
fever2.mid.npy
Gaviota trio 2.mid.npy
INVITATION.mid.npy
4thAvenueTheme.mid.npy
caravan.mid.npy
alfiepno.mid.npy
WWHappens.mid.npy
sentmood.mid.npy
This Time the Dreams on Me.mid.npy
thiscantbelove.mid.npy
bl4alice.mid.npy
Isfahan (Elf).mid.npy
ladybegood.mid.npy
YBSN2CH2.mid.npy


KeyboardInterrupt: 